In [1]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from helper.system import Hamiltonian

In [2]:
# ── System parameters (same as qenergy_vs_app.py) ──────────────────────────
Ej = 30.19
Ec = 0.1
omega_c1 = 5.226
omega_c2 = 8.135
phi_ex = 0.2
bare_dim = [10, 1, 6]
trunc_dim = [5, 1, 4]

sc = Hamiltonian(phi_ex, Ej, Ec, bare_dim, trunc_dim, omega_c1, omega_c2)

# Dressed SNAIL frequency (zero-drive reference for detuning)
position = sc.state_index((1, 0, 0), sc.original_dim)
omega_s = sc.H_dressed[position, position].real  # in angular frequency
print(f"Dressed SNAIL frequency: {omega_s / (2 * np.pi):.4f} GHz")

Dressed SNAIL frequency: 6.1591 GHz


In [39]:
# ── Sweep parameters ────────────────────────────────────────────────────────
N_amp = 100
N_half = 50  # points per side (negative + positive)
N_det = 2 * N_half

# Drive amplitudes: log-spaced between the two amplitudes from qenergy_vs_app.py
A1 = 0.04e-3 * 2 * np.pi
A2 = 5e-3 * 2 * np.pi
amplitudes = np.logspace(np.log10(A1), np.log10(A2), N_amp)

# Symmetric detuning: log-spaced in |detuning|, same range on both sides
det_max = A2*1 # GHz
det_min = A1*2  # GHz
pos_det = np.logspace(np.log10(det_min), np.log10(det_max), N_half)
neg_det = -pos_det[::-1]  # mirror: most negative first

detunings_ghz = np.concatenate([neg_det, pos_det])
detunings = detunings_ghz * 2 * np.pi  # angular frequency

# Drive frequencies = omega_s + detuning
omega_ds = omega_s + detunings

print(f"Detuning points (GHz): {np.round(detunings_ghz, 4)}")

# ── Calculation ─────────────────────────────────────────────────────────────
def calculate_static_rate(omegad, amplitude):
    der = sc.equasi_gradient(amplitude, omegad)[0]
    return np.abs(der) / 2 / np.pi

print(f"Running 2D sweep: {N_amp} amplitudes x {N_det} detunings = {N_amp * N_det} points ...")

# Flatten the 2D grid for parallel computation
tasks = [(i_amp, i_det, omega_ds[i_det], amplitudes[i_amp])
         for i_amp in range(N_amp) for i_det in range(N_det)]

results = Parallel(n_jobs=-1, verbose=5)(
    delayed(calculate_static_rate)(omegad, amp)
    for (_, _, omegad, amp) in tasks
)

# Reshape into 2D array
rates = np.zeros((N_amp, N_det))
for idx, (i_amp, i_det, _, _) in enumerate(tasks):
    rates[i_amp, i_det] = results[idx]

Detuning points (GHz): [-0.0314 -0.0289 -0.0265 -0.0244 -0.0224 -0.0206 -0.0189 -0.0174 -0.016
 -0.0147 -0.0135 -0.0124 -0.0114 -0.0105 -0.0096 -0.0089 -0.0081 -0.0075
 -0.0069 -0.0063 -0.0058 -0.0053 -0.0049 -0.0045 -0.0041 -0.0038 -0.0035
 -0.0032 -0.003  -0.0027 -0.0025 -0.0023 -0.0021 -0.0019 -0.0018 -0.0016
 -0.0015 -0.0014 -0.0013 -0.0012 -0.0011 -0.001  -0.0009 -0.0008 -0.0008
 -0.0007 -0.0006 -0.0006 -0.0005 -0.0005  0.0005  0.0005  0.0006  0.0006
  0.0007  0.0008  0.0008  0.0009  0.001   0.0011  0.0012  0.0013  0.0014
  0.0015  0.0016  0.0018  0.0019  0.0021  0.0023  0.0025  0.0027  0.003
  0.0032  0.0035  0.0038  0.0041  0.0045  0.0049  0.0053  0.0058  0.0063
  0.0069  0.0075  0.0081  0.0089  0.0096  0.0105  0.0114  0.0124  0.0135
  0.0147  0.016   0.0174  0.0189  0.0206  0.0224  0.0244  0.0265  0.0289
  0.0314]
Running 2D sweep: 100 amplitudes x 100 detunings = 10000 points ...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 322 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 520 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 754 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 1330 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 1672 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 2050 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 2464 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done 2914 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 3400 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done 3922 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 4480 tasks      | elapsed:   28.8s
[Parallel(n_jobs=-1)]: Done 5074 tasks      | 

In [40]:
# ── Plotting ────────────────────────────────────────────────────────────────
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'Times', 'DejaVu Serif'],
    'font.size': 16,
    'axes.titlesize': 16,
    'axes.labelsize': 20,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 16,
    'figure.titlesize': 16,
    'mathtext.fontset': 'stix'
})

fig, ax = plt.subplots(figsize=(8, 6))

# Undriven rate: use smallest amplitude as baseline (first row)
undriven_rate = rates[0, :]  # shape (N_det,)
ratio = rates / undriven_rate[np.newaxis, :]  # driven / undriven

# Use integer indices for x-axis so all pixels are equal width (log-like labeling)
x_idx = np.arange(N_det)
X, Y = np.meshgrid(x_idx, amplitudes / (2 * np.pi) * 1e3)

pcm = ax.pcolormesh(
    X, Y, ratio,
    shading='nearest',
    norm=plt.matplotlib.colors.LogNorm(),
    cmap='inferno',
)

# Custom x-axis tick labels: show actual detuning values with sign
# Show every other tick to avoid crowding
step = 2
tick_positions = x_idx[::step]
tick_labels = []
for d in detunings_ghz[::step]:
    if abs(d) >= 0.01:
        tick_labels.append(f'{d:.2f}')
    else:
        tick_labels.append(f'{d:.3f}')
ax.set_xticks(tick_positions)
ax.set_xticklabels(tick_labels, rotation=45, ha='right')

# Add a vertical line at the boundary between negative and positive detuning
ax.axvline(x=N_half - 0.5, color='white', linestyle='--', linewidth=0.8, alpha=0.5)

ax.set_xlabel(r'Detuning $(\omega_d - \bar{\omega}_b)/2\pi$ (GHz)')
ax.set_ylabel(r'Drive amplitude $\Omega_0/2\pi$ (MHz)')
ax.set_yscale('log')

cbar = fig.colorbar(pcm, ax=ax, label=r'Driven / Undriven ratio')

plt.tight_layout()
plt.savefig('qenergy_2d_sweep.pdf', bbox_inches='tight')
plt.show()